In [1]:
from sqlalchemy import MetaData, Table, Column
from sqlalchemy import String, Integer, ForeignKey

metadata = MetaData()

user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50)),
    Column("fullname", String(50)),
)

address_table = Table(
    "email_address",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String(100), nullable=False),
)

from sqlalchemy import create_engine

engine = create_engine("sqlite://", echo=True, future=True)
with engine.begin() as conn:
    metadata.create_all(conn)

with engine.begin() as connection:
    connection.execute(
        user_table.insert(),
        [
            {"user_id": 1, "username": "spongebob", "fullname": "Spongebob Squarepants"},
            {"user_id": 2, "username": "sandy", "fullname": "Sandy Cheeks"},
            {"user_id": 3, "username": "patrick", "fullname": "Patrick Star"},
        ],
    )

    connection.execute(
        address_table.insert(),
        [
            {"user_id": 1, "email_address": "spongebob@spongebob.com"},
            {"user_id": 1, "email_address": "spongebob@gmail.com"},
            {"user_id": 2, "email_address": "sandy@yahoo.com"},
            {"user_id": 3, "email_address": "patrick@gmail.com"},
        ],
    )

2023-06-24 14:01:29,729 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 14:01:29,730 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-24 14:01:29,730 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-24 14:01:29,731 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-24 14:01:29,731 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-24 14:01:29,732 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("email_address")
2023-06-24 14:01:29,732 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-24 14:01:29,733 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("email_address")
2023-06-24 14:01:29,733 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-24 14:01:29,734 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	username VARCHAR(50), 
	fullname VARCHAR(50), 
	PRIMARY KEY (id)
)


2023-06-24 14:01:29,734 INFO sqlalchemy.engine.Engine [no key 0.00035s] ()
2023-06-24 14:01:29,735 INFO sqlalch

In [4]:
from sqlalchemy import select

connection = engine.connect()

stmt = select(user_table.c.username, address_table.c.email_address)
print(stmt)

SELECT user_account.username, email_address.email_address 
FROM user_account, email_address


In [5]:
# Will give a warning because cartesian product can crush memory.
result = connection.execute(stmt)

2023-06-24 14:01:43,358 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-24 14:01:43,359 INFO sqlalchemy.engine.Engine SELECT user_account.username, email_address.email_address 
FROM user_account, email_address
2023-06-24 14:01:43,359 INFO sqlalchemy.engine.Engine [cached since 8.929s ago] ()


In [6]:
result.all()

[('spongebob', 'spongebob@spongebob.com'),
 ('spongebob', 'spongebob@gmail.com'),
 ('spongebob', 'sandy@yahoo.com'),
 ('spongebob', 'patrick@gmail.com'),
 ('sandy', 'spongebob@spongebob.com'),
 ('sandy', 'spongebob@gmail.com'),
 ('sandy', 'sandy@yahoo.com'),
 ('sandy', 'patrick@gmail.com'),
 ('patrick', 'spongebob@spongebob.com'),
 ('patrick', 'spongebob@gmail.com'),
 ('patrick', 'sandy@yahoo.com'),
 ('patrick', 'patrick@gmail.com')]

In [7]:
# Join tables.
# Foreign keys will be assumed as the column to join.
stmt = select(
    user_table.c.username, address_table.c.email_address
).join_from(user_table, address_table)

connection.execute(stmt).all()

2023-06-24 14:03:24,375 INFO sqlalchemy.engine.Engine SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id
2023-06-24 14:03:24,376 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()


[('spongebob', 'spongebob@spongebob.com'),
 ('spongebob', 'spongebob@gmail.com'),
 ('sandy', 'sandy@yahoo.com'),
 ('patrick', 'patrick@gmail.com')]

In [17]:
# Infer the left-most table.
stmt = select(
    user_table.c.username, address_table.c.email_address
).join(address_table)

connection.execute(stmt).all()

2023-06-24 14:12:00,966 INFO sqlalchemy.engine.Engine SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id
2023-06-24 14:12:00,967 INFO sqlalchemy.engine.Engine [cached since 123.7s ago] ()


[('spongebob', 'spongebob@spongebob.com'),
 ('spongebob', 'spongebob@gmail.com'),
 ('sandy', 'sandy@yahoo.com'),
 ('patrick', 'patrick@gmail.com')]

In [18]:
# Express the join condition explicitly.
stmt = select(
    user_table.c.username, address_table.c.email_address
).join_from(user_table, address_table, user_table.c.id == address_table.c.user_id)

stmt = select(
    user_table.c.username, address_table.c.email_address
).join(address_table, user_table.c.id == address_table.c.user_id)

connection.execute(stmt).all()

2023-06-24 14:12:01,641 INFO sqlalchemy.engine.Engine SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id
2023-06-24 14:12:01,642 INFO sqlalchemy.engine.Engine [cached since 109.7s ago] ()


[('spongebob', 'spongebob@spongebob.com'),
 ('spongebob', 'spongebob@gmail.com'),
 ('sandy', 'sandy@yahoo.com'),
 ('patrick', 'patrick@gmail.com')]

In [19]:
address_alias_1 = address_table.alias()
address_alias_2 = address_table.alias()

select_stmt = (
    select(
            user_table.c.username,
            address_alias_1.c.email_address,
            address_alias_2.c.email_address,
    )
    .join_from(user_table, address_alias_1)
    .join_from(user_table, address_alias_2)
    .where(address_alias_1.c.email_address == "spongebob@spongebob.com")
    .where(address_alias_2.c.email_address == "spongebob@gmail.com")
)

connection.execute(select_stmt).all()

2023-06-24 14:30:14,923 INFO sqlalchemy.engine.Engine SELECT user_account.username, email_address_1.email_address, email_address_3.email_address AS email_address_2 
FROM user_account JOIN email_address AS email_address_1 ON user_account.id = email_address_1.user_id JOIN email_address AS email_address_3 ON user_account.id = email_address_3.user_id 
WHERE email_address_1.email_address = ? AND email_address_3.email_address = ?
2023-06-24 14:30:14,924 INFO sqlalchemy.engine.Engine [generated in 0.00138s] ('spongebob@spongebob.com', 'spongebob@gmail.com')


[('spongebob', 'spongebob@spongebob.com', 'spongebob@gmail.com')]

In [20]:
# Subquery can be used as an alias.
select_subq = (
    select(user_table.c.username, address_table.c.email_address)
    .join(address_table).subquery()
)

stmt = select(select_subq.c.username).where(
    select_subq.c.username == "spongebob"
)
print(stmt)

SELECT anon_1.username 
FROM (SELECT user_account.username AS username, email_address.email_address AS email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id) AS anon_1 
WHERE anon_1.username = :username_1


In [22]:
from sqlalchemy import func

address_select = select(
    address_table.c.user_id, func.count(address_table.c.id).label("count")
).group_by(address_table.c.user_id)

address_subq = address_select.subquery()

username_plus_count = (
    select(user_table.c.username, address_subq.c.count)
    .join(address_subq)
    .order_by(user_table.c.username)
)

connection.execute(username_plus_count).all()

2023-06-24 14:37:28,329 INFO sqlalchemy.engine.Engine SELECT user_account.username, anon_1.count 
FROM user_account JOIN (SELECT email_address.user_id AS user_id, count(email_address.id) AS count 
FROM email_address GROUP BY email_address.user_id) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.username
2023-06-24 14:37:28,330 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ()


[('patrick', 1), ('sandy', 1), ('spongebob', 2)]

In [24]:
# Common Table Expressions (CTE) similar to subqueries.
address_cte = address_select.cte()

username_plus_count = (
    select(user_table.c.username, address_cte.c.count)
    .join(address_cte)
    .order_by(user_table.c.username)
)

connection.execute(username_plus_count).all()

2023-06-24 14:47:49,029 INFO sqlalchemy.engine.Engine WITH anon_1 AS 
(SELECT email_address.user_id AS user_id, count(email_address.id) AS count 
FROM email_address GROUP BY email_address.user_id)
 SELECT user_account.username, anon_1.count 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.username
2023-06-24 14:47:49,029 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()


[('patrick', 1), ('sandy', 1), ('spongebob', 2)]

In [29]:
# Correlated subqueries will be evaluated for each row of the outer query.
address_corr = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .scalar_subquery()
)

print(address_corr)
# connection.execute(address_corr).all()


(SELECT count(email_address.id) AS count_1 
FROM email_address, user_account 
WHERE user_account.id = email_address.user_id)


In [30]:
select_stmt = select(user_table.c.username, address_corr)
print(select_stmt)
connection.execute(select_stmt).all()

SELECT user_account.username, (SELECT count(email_address.id) AS count_1 
FROM email_address 
WHERE user_account.id = email_address.user_id) AS anon_1 
FROM user_account
2023-06-24 14:54:10,547 INFO sqlalchemy.engine.Engine SELECT user_account.username, (SELECT count(email_address.id) AS count_1 
FROM email_address 
WHERE user_account.id = email_address.user_id) AS anon_1 
FROM user_account
2023-06-24 14:54:10,548 INFO sqlalchemy.engine.Engine [cached since 172.5s ago] ()


[('spongebob', 2), ('sandy', 1), ('patrick', 1)]